In [4]:
import pandas as pd 
df = pd.read_csv("data/radha_krishna_sentiment_500.csv")
X=df['text']
y=df['label']

In [5]:
%pip install pandas



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
   X,y,test_size=0.2,random_state=42

)


In [7]:
# Baseline Model (BoW + Naive Bayes)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
model_nb = MultinomialNB()
model_nb.fit(X_train_vec,y_train)
y_pred = model_nb.predict(X_test_vec)
print("Accuracy:",accuracy_score(y_pred,y_test))
print("Classification Report : ", classification_report(y_test,y_pred))#model ne har class ko kitna axa predict kiya hai class 0 -- neg classs class 1 -- pos class 

Accuracy: 1.0
Classification Report :                precision    recall  f1-score   support

    negative       1.00      1.00      1.00        47
    positive       1.00      1.00      1.00        53

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [9]:
# TF-IDF + Logistic Regression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000) #logistic regression does have direct formula to get weights it is uses gradient descent  so it computes error and resolves and repeat again and again
lr.fit(X_train_tf,y_train)
y_pred = lr.predict(X_test_tf)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00        47
    positive       1.00      1.00      1.00        53

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [11]:
#Ngram experiment TF-IDF + Bigrams
tfidf_bg = TfidfVectorizer(ngram_range=(1,2))
X_train_bg = tfidf_bg.fit_transform(X_train)
X_test_bg = tfidf_bg.transform(X_test)
lr.fit(X_train_bg,y_train)
y_pred = lr.predict(X_test_bg)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00        47
    positive       1.00      1.00      1.00        53

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [12]:
# Word2Vec
from gensim.utils import simple_preprocess
tokenized_train = [simple_preprocess(text) for text in X_train]
tokenized_test = [simple_preprocess(text) for text in X_test]


In [13]:
%pip install gensim



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gensim.models import Word2Vec
# Word2Vec learns a numerical vector for each word such that
# 👉 words used in similar contexts get similar vectors

In [15]:
w2v = Word2Vec(
    sentences=tokenized_train,
    vector_size=100, #each word will be represented by 100 numbers
    window=5, #context size
    min_count=2, #ignore words that is less than 2
    sg=1 #training architecture 0--> cbow 1-->skipgram
)

In [18]:
import numpy as np 
def doc_vector(doc,model):
    vec = np.zeros(model.vector_size)
    count = 0
    for word in doc:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    return vec/count if count !=0 else vec
    

In [19]:
X_train_w2v = np.array([doc_vector(d,w2v) for d in tokenized_train])
X_test_w2v = np.array([doc_vector(d,w2v) for d in tokenized_test])
lr.fit(X_train_w2v, y_train)
y_pred = lr.predict(X_test_w2v)
print("Word2Vec Accuracy:", accuracy_score(y_pred,y_test))

Word2Vec Accuracy: 0.61
